Goal: realized I didnt save enough station metadata in "find_all_stations", so this is a second pass for more info from the subset previously identified

Order of operations:
- load all_relevant_stations.pkl
- sort by chl|turb|sedi|par|phot|secchi|ssc
- sort those by buoy/glider/ship

In [1]:
#pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import urllib.request
import json
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [8]:
# load previous search output
FF = pd.read_pickle('all_relevant_stations.pkl')

lib = {'buoy','glider','ship','turbid','ssc','photo','chl','radiation'}

FF["buoy"] = False
FF["glider"] = False
FF["ship"] = False
FF["turbid"] = False
FF["ssc"] = False
FF["photo"] = False
FF["chl"] = False
FF['radiation'] = False
FF['bad'] = False

for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity

    # make the info URL for this site for this time range
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv"
              )
    e.dataset_id = FF['sites'][jj]
    # e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
    try:
        ds_info = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"bad"] = True
    else:
        for kwd in lib:
            idx = ds_info["Value"].str.contains(kwd, flags = re.IGNORECASE)
            if any(idx==True)==True:
                FF.loc[jj,kwd] = True

FF = FF[FF.bad == False] 
FF.reset_index(drop=True, inplace=True)
FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])

FF = FF.drop(columns="bad")
FF.to_pickle("labeled_relevant_stations.pkl")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700


In [10]:
print(len(FF[(FF.buoy == True) & (FF.radiation == True)]))
FF[(FF.buoy == True) & (FF.radiation == True)]

42


,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units,check_sum,buoy,glider,ship,turb,ssc,phot,chl,turbid,photo,radiation
354,mpoPmzaVikingBuoysTimeseries,https://erddap.ogsl.ca/erddap/,2020-07-09T05:00:00Z,2024-07-01T23:00:00Z,44.3493,49.7347,-124.7715,-52.5862,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
395,bcodmo_dataset_737163,https://erddap.bco-dmo.org/erddap/,1989-07-29T19:00:00Z,NaN,22.75,22.75,-158.0,-158.0,degrees_north,degrees_east,1.0,True,False,True,True,False,True,True,False,True,True
396,bcodmo_dataset_3093,https://erddap.bco-dmo.org/erddap/,1995-11-08T10:12:00Z,NaN,10.492,10.683,-64.735,-64.367,degrees_north,degrees_east,1.0,True,False,True,True,False,True,True,True,True,True
533,edu_humboldt_humboldt,https://erddap.sensors.ioos.us/erddap/,2013-02-14T17:15:00Z,2024-07-01T21:45:00Z,40.7775,40.7775,-124.19652,-124.19652,degrees_north,degrees_east,1.0,True,False,False,True,False,True,True,True,True,True
563,nerrs_kachomet,https://erddap.sensors.ioos.us/erddap/,2003-06-01T09:15:00Z,2024-05-28T21:00:00Z,59.60093,59.60093,-151.4095,-151.4095,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
666,ism-glos-obs_46,https://erddap.sensors.ioos.us/erddap/,2023-05-05T18:15:00Z,2024-07-01T18:20:00Z,46.96873,46.96873,-88.39768,-88.39768,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
815,edu_humboldt_humboldt,http://erddap.cencoos.org/erddap/,2013-02-14T17:15:00Z,2024-07-01T23:45:00Z,40.7775,40.7775,-124.19652,-124.19652,degrees_north,degrees_east,1.0,True,False,False,True,False,True,True,True,True,True
909,bios_jack-20190725T1346-delayed,https://gliders.ioos.us/erddap/,2019-07-25T13:58:08Z,2019-08-17T16:15:26Z,31.615135581954483,32.359756468666305,-64.57020576961442,-63.86904867873203,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True
910,bios_jack-20190917T0241-delayed,https://gliders.ioos.us/erddap/,2019-09-17T02:09:41Z,2019-10-21T09:47:37Z,31.664384917652953,31.86234156782121,-64.16728851495773,-63.887413503596065,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True
920,bios_jack-20220627T1351-delayed,https://gliders.ioos.us/erddap/,2022-06-27T13:27:55Z,2022-08-19T15:30:24Z,32.00741863035957,32.36251482479653,-64.59722646073882,-63.83851135028313,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True
